# pH Implementation

##### Author: Hrönn Kjartansdóttir
##### Methanogenesis model where pH has been implemented

In [ ]:
# Import all packages needed for the model
import numpy as np
import scipy
from scipy.integrate import solve_ivp
from scipy.optimize import least_squares

import matplotlib.pyplot as plt
import pandas as pd

import random
import math

import os
os.chdir('C:\\Users\\hronn\\Desktop\\Jupyter')

In [ ]:
# Import the experimental data -> Measured pressure values over time
df = pd.read_csv("reactor1\\data\\reactor1_20H2_days178_213.csv", sep=",")
df

In [ ]:
# Sort the experimental data to get arrays of:
    # df_indices = indices of experimental pressure values grouped by fills -> df_indices[0] = indices for the first fill
    # nan_indices = nan values grouped to determine where the margin between fills lies
    # df_pressure = experimental pressure values grouped by fills
    # in_pressure = starting pressure for each fill
    # end_pressure = end pressure value for each fill
    # df_time = time each pressure value was measured grouped by fills

def dataframe(dataframe, time):
            #dataframe indexes for NaN's to list
    df = dataframe.index.get_indexer(dataframe.index[dataframe.notnull()]).tolist()
    NaN = dataframe.index.get_indexer(dataframe.index[dataframe.isnull()]).tolist()
    
            #Group the indexes of NaN's for the dataframes pressure values -> where the shift between fills occurs
    def indices(values):
        
        prev_number = min(values) if values else None
        indices = list()

        for number in sorted(values):
            if number != prev_number+1:
                indices.append([number])
            elif len(indices[-1]) > 1:
                indices[-1][-1] = number
            else:
                indices[-1].append(number)
            prev_number = number
            
        return indices
    
    df_indices = indices(df)
    nan_indices = indices(NaN)
    
    df_pressure = []

    def values(dataframe):
        for i in df_indices:
        
            values = dataframe.iloc[i[0]:(i[1]+1)].tolist()
            df_pressure.append(values)
            
        return df_pressure
    
    df_pressure = values(dataframe) 
    
    in_pres = []
    end_pres = []
    
    for i in range(len(df_pressure)):
    
        inn = df_pressure[i][0]
        out = df_pressure[i][-1]
    
        in_pres.append(inn)
        end_pres.append(out)
    
    df_time = []
    
    for i in df_indices:
        time_values = time.iloc[i[0]:(i[1]+1)].tolist()
        df_time.append(time_values)

    
    return df_indices, nan_indices, df_pressure, in_pres, end_pres, df_time

[df_indices, nan_indices, df_pressure, in_pressure, end_pressure, df_time ] = dataframe(df["pressure_pascal"], df["days"])



In [ ]:
        #Initial mole fraction:
x_g_H2_th = 0.20 # *100 = % # <- Change accordingly
x_g_CO2_th = 0.05 # *100 = % # <- Change accordingly


In [ ]:
# End values from 10% H2 and 2.5% CO2 from previous fill before concentration was changed to 20% H2 and 5% CO2

w_l_CH4_in = 2.44e-04 # <- Change accordingly
S_liq_in = 3.60e-01 # <- Change accordingly
BM_in = 87.95 # <- Change accordingly


In [ ]:
# Set all global variables for the model

        # Molecular weight of liquids in mol/m3 = mol/1000L:
M_H2  = 2.016*1e-3 
M_CO2 = 44.009*1e-3
M_CH4 = 16.043*1e-3
M_H2O = 18.015*1e-3

        #Liquid density:
density_l = 992.2 # kg/m³

        #Sandrock porosity:
pore = 0.2 # %

        # Dissolution/evaporation rate constants:
K_diss_H2 = 1.05e-2
K_diss_CO2 = 1.87e-3
K_diss_CH4 = 9.32e-5

K_evap_CH4 = 1.81e-2

        # Maximum biomass growth rate:
mu_max = 0.0427 # 1/s

        # temperature:
temperature = 273+40 # Kelvin (K)

        #max uptake rates:
r_H2_max = 1 # mmol/g*h
r_CH4_min = 0 # mmol/g*h
r_CO2_max = r_H2_max / 4 # mmol/g*h

        #Henry solubility constants:
H_H2 = 7.13e-9*1e3
H_CO2 = 2.31e-7*1e3
H_CH4 = 1.08e-8*1e3

#Dissociation constants of Carbonic acid
k1 = 5.105e-7 #at 40°C and Salinity of 0‰
k2 = 0.617e-10 #at 40°C and Salinity of 0‰

        #Pressure in reactor at end of evaporation phase
pressureCH4resid = 2.06e5


In [ ]:
#### This RK45 tracks in this notebook only sigma as parameter of interest

def rk45(t, y, sigma):
    w_g_H2, w_g_CO2, w_g_CH4, w_l_H2, w_l_CO2, w_l_CH4, m, density_g, S_liq = y
    
    
            #molecular weight of gasses:
    M_g = ((w_g_H2/M_H2) + (w_g_CO2/M_CO2) + (w_g_CH4/M_CH4))**-1
    

            #gas pressure:
    R_gas = 8.314 / M_g
    pressure_g = density_g * R_gas * temperature

              #mole fractions of gas phase:
    x_g_H2 = (w_g_H2/M_H2)*M_g
    x_g_CO2 = (w_g_CO2/M_CO2)*M_g
    x_g_CH4 = (w_g_CH4/M_CH4)*M_g


            #mass fraction of liquid H2O:
    w_l_H2O = 1 - (w_l_H2 + w_l_CO2 + w_l_CH4)

            #Mass(molecular weight) of all liquids
    M_l = ((w_l_H2/M_H2) + (w_l_CO2/M_CO2) + (w_l_CH4/M_CH4) + (w_l_H2O/M_H2O))**-1


            #mole fractions of liquid phase:
    x_l_H2 = (w_l_H2/M_H2)*M_l
    x_l_CO2 = (w_l_CO2/M_CO2)*M_l
    x_l_CH4 = (w_l_CH4/M_CH4)*M_l

            #saturated mole fraction
    x_l_sat_H2 = (M_l/density_l) * H_H2 * x_g_H2 * pressure_g
    x_l_sat_CO2 = (M_l/density_l) * H_CO2 * x_g_CO2 * pressure_g
    x_l_sat_CH4 = (M_l/density_l) * H_CH4 * x_g_CH4 * pressure_g
       
       
            #source term due to dissolution/evaporation
    R_diss_H2  = K_diss_H2*(x_l_sat_H2 - x_l_H2)
    R_diss_CO2 = K_diss_CO2*(x_l_sat_CO2 - x_l_CO2)

    if x_l_CH4 < x_l_sat_CH4:
        R_diss_CH4 = K_diss_CH4*(x_l_sat_CH4 - x_l_CH4)
    else:
        R_diss_CH4 = K_evap_CH4*(x_l_sat_CH4 - x_l_CH4)


    R_diss_tot = R_diss_H2 + R_diss_CO2 + R_diss_CH4

            #Source for saturation of liquid phase
    source_S_liq = (1/(pore*density_l))*R_diss_tot
       
            #saturation of gas phase
    S_gas = 1 - S_liq
    
    
            # Calculate pH:
    #####################################
    

    U_CO2 = 10**5.08
    K_Monod_H2 = 10**-6.09
    K_Monod_CO2 = 10**-6.12
    d = 10**-4.19

    k1 = 5.105e-7 #at 40°C and Salinity of 0‰
    density_l_new = 0.9922 #kg/L
    
    c_CO2 = (x_l_CO2 * density_l_new) / M_l # CO2 concentration in mol/L
    
    
    if c_CO2 < 1.96 * 1e-8:
        pH = 7   
    else:
        pH = -np.log10((k1 * c_CO2)**0.5)  
    
  
    factor = sigma**2 / ( (pH-7.5)**2 + sigma**2)
            
            #uptake/production rates of H2, CO2, CH4 (r_(i)):
    r_CO2  = min(U_CO2 * x_l_CO2 , r_CO2_max) * factor #mol of CO2 / kg of dry biomass / hour
    mu_monod = mu_max * (x_l_H2 / (K_Monod_H2 + x_l_H2)) * (x_l_CO2 / (K_Monod_CO2 + x_l_CO2))
    mu = min((mu_monod  * factor), (r_CO2 * mu_max / r_CO2_max))
        
    
    r_H2 = r_CO2 * 4
    r_CH4 = r_CO2 - (5.855 * mu)

            #Biomass change over time
    source_m = ((mu - d) * m)/3600 # divide by 3600 to get all units in si units (from hour to seconds)

            #Source term for production/uptake
    R_uptake_H2 = (-r_H2*m*M_H2*S_liq*pore)/3600
    R_uptake_CO2 = (-r_CO2*m*M_CO2*S_liq*pore)/3600
    R_uptake_CH4 = (r_CH4*m*M_CH4*S_liq*pore)/3600


            #Source of mass fractions change over time
    source_w_l_H2 = 1/(pore*density_l*S_liq) * (R_uptake_H2 + R_diss_H2 - (w_l_H2 * R_diss_tot))
    source_w_l_CO2 = 1/(pore*density_l*S_liq) * (R_uptake_CO2 + R_diss_CO2 - (w_l_CO2 * R_diss_tot))
    source_w_l_CH4 = 1/(pore*density_l*S_liq) * (R_uptake_CH4 + R_diss_CH4 - (w_l_CH4 * R_diss_tot))

            #gas density change
    source_density_g = -(R_diss_tot / (pore * S_gas)) * (1 - (density_g / density_l))
        
             #mass fraction change 
    source_w_g_H2 = -(1/(pore*density_g*S_gas)) * (R_diss_H2 - w_g_H2 * R_diss_tot) 
    source_w_g_CO2 = -(1/(pore*density_g*S_gas)) * (R_diss_CO2 - w_g_CO2 * R_diss_tot) 
    source_w_g_CH4 = -(1/(pore*density_g*S_gas)) * (R_diss_CH4 - w_g_CH4 * R_diss_tot) 
    

    return [source_w_g_H2, source_w_g_CO2, source_w_g_CH4, source_w_l_H2, source_w_l_CO2, source_w_l_CH4, source_m, source_density_g, source_S_liq]

In [ ]:
# If needing to check if time data is sorted correctly -> if not go back to experimental data and look for where the error occurred

            # Time data to test:
test_list = df_time[0]
   
            # Check sorting of the list using sort()
flag = 0
test_list1 = test_list[:]
test_list1.sort()
if (test_list1 == test_list):
    flag = 1
      
            # print the results
if (flag) :
    print ("Yes, List is sorted.")
else :
    print ("No, List is not sorted.")

In [ ]:
# Pressure generation function - in this notebook only one function is needed for pressure simulations
def pressure_simulation(time, in_pressure, theta0, w_l_CH4_in_new, S_liq_in, BM_in_new):
    
                #Initial mass fraction of liquids:
    w_l_H2_in  = 0
    w_l_CO2_in = 0
    
            #Initial mole fraction of gas phase:
    x_g_H2_in  = (in_pressure - pressureCH4resid) / in_pressure * x_g_H2_th
    x_g_CO2_in = (in_pressure - pressureCH4resid) / in_pressure * x_g_CO2_th 
    x_g_CH4_in = 1 - x_g_H2_in - x_g_CO2_in
   
            #Initial molecular mass of all gasses:
    M_g = (x_g_H2_in*M_H2) + (x_g_CO2_in*M_CO2) + (x_g_CH4_in*M_CH4)

            #Initial gas pressure:
    R_gas_in = 8.314/M_g
    density_g_in = in_pressure/R_gas_in/temperature

        #Initial mass fraction of gasses:
    w_g_H2_in  = (x_g_H2_in * M_H2) / M_g
    w_g_CO2_in = (x_g_CO2_in * M_CO2) / M_g
    w_g_CH4_in = (x_g_CH4_in * M_CH4) / M_g
    
        # Time array in seconds and end time:
    time_sim = np.array(time) - np.array(time[0])
    time_end = time_sim[-1]
    time_sim = time_sim*24*60*60               

        # Initial value for sigma
    sigma_in = 10**theta0[0]
    
        # Solve_ivp to solve for RK45():
    sol = solve_ivp(rk45, [0,time_end],y0=[w_g_H2_in, w_g_CO2_in, w_g_CH4_in, w_l_H2_in, w_l_CO2_in, w_l_CH4_in_new, BM_in_new, density_g_in, S_liq_in], 
                    args= [sigma_in], t_eval=time_sim, atol=1e-8, rtol=1e-6)
   
        # Save the tracked and returned values from solve_ivp():
    w_g_H2 = sol.y[0]
    w_g_CO2 = sol.y[1]
    w_g_CH4 = sol.y[2]
    w_l_H2 = sol.y[3] 
    w_l_CO2 = sol.y[4]
    w_l_CH4 = sol.y[5]
    biomass = sol.y[6]
    density_g = sol.y[7]
    S_liq = sol.y[8]

          # Calculate the new initial saturation of liquid phase to use as input for the next fill:
    S_liq_in_new = S_liq[-1] * (1- (1-0.557)*w_l_CH4[-1]-w_l_H2[-1]-w_l_CO2[-1])
    
            # Calculate the new initial molecular weight of gasses:
    M_g_new = ((w_g_H2/M_H2) + (w_g_CO2/M_CO2) + (w_g_CH4/M_CH4))**-1
    
            # Calculate the pressure change over time:
    R_gas = 8.314 / M_g_new
    pressure_g_bar = density_g * R_gas * temperature * 1e-5 #to change from pascal to bar * 1e-5
    pressure_g_pascal = density_g * R_gas * temperature

            # Calculate the initial mass fraction of CH4 to input into next fill:
    w_l_CH4_in_new = w_l_CH4[-1]*0.557 # 55.7% of the end value go to the next fill
    
            # Calculate the initial biomass to input into next fill:
    BM_in_new = biomass[-1] /(1- (1-0.557)*w_l_CH4[-1]-w_l_H2[-1]-w_l_CO2[-1])

            # Variables to be returned:
    return w_l_CH4_in_new, pressure_g_pascal, S_liq_in_new, BM_in_new, biomass


In [ ]:
# Setting the parameter of interest: sigma as a fixed initial guess value, theta0
theta0 = [-5]


In [ ]:
#Generate pressure_g_sim -> simulated pressure

def simulated_pressure(df_time, theta0, w_l_CH4_in, S_liq_in, BM_in):
    pressure = []
    biomass = []
    w_l_CH4_end_all = []
    S_liq_end_all = []
    
        
    for i in range(len(df_pressure)):
        [w_l_CH4_in, pressure_g_sim_fills, S_liq_in, BM_in, biomass_fills] = pressure_simulation(df_time[i], in_pressure[i], theta0, w_l_CH4_in, S_liq_in, BM_in)
        pressure.append(pressure_g_sim_fills)
        biomass.append(biomass_fills)
        w_l_CH4_end_all.append(w_l_CH4_in)
        S_liq_end_all.append(S_liq_in)

    biomass_concatenated = np.concatenate(biomass[0:len(biomass)])
    biomass_endvalue = biomass_concatenated[-1]
    
 
    w_l_CH4_endvalue = w_l_CH4_end_all[-1]
    
    S_liq_endvalue = S_liq_end_all[-1]
    
    return pressure, biomass, biomass_endvalue, biomass_concatenated, w_l_CH4_endvalue, S_liq_endvalue
    
    
[pressure_g_sim, biomass, biomass_endvalue, biomass_concatenated, w_l_CH4_endvalue, S_liq_endvalue] = simulated_pressure(df_time, theta0, w_l_CH4_in, S_liq_in, BM_in)

In [ ]:
# End values for biomass , w_l_CH4 , and S_liq for the last fill of this gas composition

print("Biomass end value:", round(biomass_endvalue, 2))
print("w_l_CH4 end value:", "{:.2e}".format(w_l_CH4_endvalue))
print("S_liq end value:", "{:.2e}".format(S_liq_endvalue))

In [ ]:
#Plot pressure comparison between simulated pressure and experimental pressure
def plot_pressure_comparison(df_time, df_pressure, pressure_g_sim):

    fig, (ax1) = plt.subplots(figsize=(12,6))
    
    ax1.plot(df_time[0], pressure_g_sim[0], color="brown", label='Simulated Pressure')
    ax1.plot(df_time[0], df_pressure[0], color="cornflowerblue", label='Experimental Pressure')
    ax1.legend(loc="upper right")
    ax1.set_xlabel("Days")
    ax1.set_ylabel("Pressure [Pa]")
    
    for i in range(1, len(pressure_g_sim)):

            ax1.plot(df_time[i], pressure_g_sim[i], color="brown", label='_nolegend_')

            ax1.plot(df_time[i], df_pressure[i], color="cornflowerblue", label='_nolegend_')

    



    ax1.set_title("CO2 Model - 20% H2 and 5% CO2 - Pressure Comparison")
    
plot_pressure_comparison(df_time, df_pressure, pressure_g_sim)

In [ ]:
# Plot biomass change over time
def plot_biomass(df_time, biomass):

    fig, (ax1) = plt.subplots(figsize=(12,6))
    
    ax1.plot(df_time[0], biomass[0], color="green", label='Biomass')

    ax1.legend(loc="lower right")
    ax1.set_xlabel("Days")
    ax1.set_ylabel("Biomass")
    
    for i in range(1, len(biomass)):

            ax1.plot(df_time[i], biomass[i], color="green", label='_nolegend_')


    ax1.set_title("20% H2 and 5% CO2 days 178-213 - Biomass")
    
plot_biomass(df_time, biomass)

In [ ]:
# Generate the pressure difference between experimental and simulated values

def pressure_diff(theta0):

    [pressure_g_sim, biomass, biomass_endvalue, biomass_concatenated, w_l_CH4_endvalue, S_liq_endvalue] = simulated_pressure(df_time, theta0, w_l_CH4_in, S_liq_in, BM_in)
    
    sim_pressure = np.concatenate(pressure_g_sim[0:len(pressure_g_sim)])
    exp_pressure = np.concatenate(df_pressure[0:len(df_pressure)])
    
    diff = exp_pressure - sim_pressure

    return diff

#pressure_g_diff = pressure_diff(theta0)

In [ ]:
# Look for optimal value for the random theta0 value and generate norm of residuals for the optimization

     # Set upper and lower bounds for the random theta0 values
l_bound = [-10]
u_bound = [10]

def optimal_values(theta0):
    
 
    res = least_squares(pressure_diff, theta0, bounds=[l_bound, u_bound])
    optimal_value = res.x
    
    residuals = res.fun
    norm_res = np.linalg.norm(residuals)
    
    return optimal_value, norm_res

[optimal_value, norm_res] = optimal_values(theta0)

In [ ]:
print("Found optimal value: ", optimal_value)
print("Norm of residuals :", norm_res)

In [ ]:
# Set theta0 = optimized value -> then you can go up and plot simulations with optimal value in plot_pressure_comparison()"
theta0 = [optimal_value[0]]